In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install open-clip-torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 75.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.3 MB/s eta 0:00:00


In [3]:
%cp "/content/drive/MyDrive/DATS 5980/aokvqa.zip" /content/

In [4]:
%cp "/content/drive/MyDrive/DATS 5980/coco_filtered.zip" /content/

In [5]:
%cd /content/

/content


In [6]:
!unzip /content/aokvqa.zip

Archive:  /content/aokvqa.zip
   creating: aokvqa/
  inflating: aokvqa/aokvqa_v1p0_test.json  
  inflating: aokvqa/aokvqa_v1p0_train.json  
  inflating: aokvqa/aokvqa_v1p0_val.json  
  inflating: aokvqa/large_vocab_train.csv  
  inflating: aokvqa/specialized_vocab_train.csv  


In [7]:
!unzip /content/coco_filtered.zip

Streaming output truncated to the last 5000 lines.
  inflating: coco_filtered/test2017/000000201802.jpg  
  inflating: coco_filtered/test2017/000000201812.jpg  
  inflating: coco_filtered/test2017/000000201975.jpg  
  inflating: coco_filtered/test2017/000000202014.jpg  
  inflating: coco_filtered/test2017/000000202038.jpg  
  inflating: coco_filtered/test2017/000000202208.jpg  
  inflating: coco_filtered/test2017/000000202294.jpg  
  inflating: coco_filtered/test2017/000000202314.jpg  
  inflating: coco_filtered/test2017/000000202435.jpg  
  inflating: coco_filtered/test2017/000000202555.jpg  
  inflating: coco_filtered/test2017/000000202625.jpg  
  inflating: coco_filtered/test2017/000000202758.jpg  
  inflating: coco_filtered/test2017/000000202771.jpg  
  inflating: coco_filtered/test2017/000000202930.jpg  
  inflating: coco_filtered/test2017/000000203394.jpg  
  inflating: coco_filtered/test2017/000000203508.jpg  
  inflating: coco_filtered/test2017/000000203550.jpg  
  inflating: c

In [8]:
import os
import json
from tqdm import tqdm
import torch
import open_clip
from PIL import Image
from torchvision import transforms
from pathlib import Path

In [9]:
import random
import numpy as np
import torch

def set_seed(seed: int = 42):
    """
    Set random seed for reproducibility across Python, NumPy, and PyTorch.
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    # For deterministic behavior (slower but fully reproducible)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

In [10]:
aokvqa_dir = os.getenv('AOKVQA_DIR', "/content/aokvqa")
coco_filtered_dir = os.getenv('COCO_FILTERED_DIR', r"/content/coco_filtered")

In [11]:
def load_aokvqa(aokvqa_dir, split, version='v1p0'):
    assert split in ['train', 'val', 'test', 'test_w_ans']
    dataset = json.load(open(
        os.path.join(aokvqa_dir, f"aokvqa_{version}_{split}.json")
    ))
    return dataset

def get_coco_path(split, image_id, coco_dir):
    return os.path.join(coco_dir, f"{split}2017", f"{image_id:012}.jpg")


In [12]:
val_json_path = "/content/aokvqa/aokvqa_v1p0_val.json"
with open(val_json_path, 'rb') as fp:
    data = json.load(fp)

In [13]:
questions = questions = [item['question'] for item in data]
choices = [item['choices'] for item in data]
image_ids = [item['image_id'] for item in data]
len(questions), len(choices), len(image_ids)

(1145, 1145, 1145)

In [14]:
images = [get_coco_path('val', image_id, coco_filtered_dir) for image_id in image_ids]
len(images)

1145

In [15]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [16]:
model, _, preprocess = open_clip.create_model_and_transforms(
    'ViT-L-14', pretrained='openai'
)
tokenizer = open_clip.get_tokenizer('ViT-L-14')
model = model.to(device)
model.eval()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


open_clip_model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/open_clip/factory.py:450: UserWarning: QuickGELU mismatch between final model config (quick_gelu=False) and pretrained tag 'openai' (quick_gelu=True).
  warnings.warn(


CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
    (patch_dropout): Identity()
    (ln_pre): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): ModuleList(
        (0-23): 24 x ResidualAttentionBlock(
          (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=1024, out_features=1024, bias=True)
          )
          (ls_1): Identity()
          (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=1024, out_features=4096, bias=True)
            (gelu): GELU(approximate='none')
            (c_proj): Linear(in_features=4096, out_features=1024, bias=True)
          )
          (ls_2): Identity()
        )
      )
    )
    (ln_post): LayerNorm((1024,), eps=1e-05, elementwi

In [17]:
# image_directory = "/content/coco_filtered/val2017"
# img_dir = Path("./images")  # change to your folder
# image_paths = sorted(list(img_dir.glob("*.jpg"))) # limit or remove [:100]

In [18]:
import re
import spacy
nlp = spacy.load("en_core_web_sm")

def extract_subject(question: str):
    """Roughly extract the noun phrase or main object of the question."""
    doc = nlp(question)
    # Prefer object of preposition or direct object
    for tok in doc:
        if tok.dep_ in ("pobj", "dobj", "nsubj", "attr"):
            span = " ".join([t.text for t in tok.subtree])
            if len(span.split()) > 1:
                return span
    # fallback to last noun chunk
    noun_chunks = list(doc.noun_chunks)
    if noun_chunks:
        return noun_chunks[-1].text
    return question

def question_to_statements(question, choices):
    q = question.lower().strip().rstrip("?")
    sents = []

    subj = extract_subject(question)

    # 1️⃣ what is / what are
    if re.search(r"\bwhat (is|are)\b", q):
        if "in " in q:
            container = q.split("in",1)[-1].strip()
            for c in choices:
                sents.append(f"{container} has {c} in it")
        elif "on " in q:
            container = q.split("on",1)[-1].strip()
            for c in choices:
                sents.append(f"{container} has {c} on it")
        elif "made of" in q or "made from" in q:
            target = q.split("made")[0].replace("what is","").strip()
            for c in choices:
                sents.append(f"{target} is made of {c}")
        elif "used for" in q:
            target = q.split("used")[0].replace("what is","").strip()
            for c in choices:
                sents.append(f"{target} is used for {c}")
        else:
            for c in choices:
                sents.append(f"{subj} is {c}")
        return sents

    # 2️⃣ what kind/type
    if "what kind" in q or "what type" in q:
        for c in choices:
            sents.append(f"{subj} is a type of {c}")
        return sents

    # 3️⃣ where / location questions
    if q.startswith("where") or "where" in q:
        for c in choices:
            sents.append(f"{subj} is located in {c}")
        return sents

    # 4️⃣ who questions
    if q.startswith("who") or "who" in q:
        for c in choices:
            sents.append(f"{subj} is {c}")
        return sents

    # 5️⃣ why / purpose questions
    if q.startswith("why") or "why" in q:
        for c in choices:
            sents.append(f"{subj} is because of {c}")
        return sents

    # 6️⃣ which number / how many
    if "which number" in q or "how many" in q:
        for c in choices:
            sents.append(f"The number is {c}")
        return sents

    # 7️⃣ what color / appearance
    if "what color" in q or "what colour" in q or "what pattern" in q:
        for c in choices:
            sents.append(f"{subj} is {c} in color")
        return sents

    # 8️⃣ what country / nationality
    if "what country" in q:
        for c in choices:
            sents.append(f"{subj} belongs to {c}")
        return sents

    # 9️⃣ what is the person doing
    if re.search(r"what.*doing", q):
        for c in choices:
            sents.append(f"{subj} is doing {c}")
        return sents

    # 🔟 fallback: generic declarative pattern
    for c in choices:
        sents.append(f"{subj} is {c}")
    return sents


In [19]:
import re

def remove_wh_questions(text):
    """
    Removes WH-question words (who, what, when, where, why, how, etc.) from the input text.
    """
    # Define common WH-question keywords
    wh_words = [
        r'\bwho\b', r'\bwhom\b', r'\bwhose\b',
        r'\bwhat\b', r'\bwhich\b', r'\bwhen\b',
        r'\bwhere\b', r'\bwhy\b', r'\bhow\b'
    ]

    # Create a regex pattern that matches any WH-word, case-insensitive
    pattern = re.compile('|'.join(wh_words), re.IGNORECASE)

    # Replace matched words with empty string
    cleaned_text = pattern.sub('', text)

    # Clean up extra spaces
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()

    return cleaned_text


In [21]:
labels = []
for idx, question in enumerate(questions):
    result = question_to_statements(question, choices[idx])
    result = [remove_wh_questions(s) for s in result]
    labels.append(result)
    print(idx, end=",")

0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,27

In [22]:
len(labels)

1145

In [23]:
labels[0]

["the motorcyclist's mouth has toothpick in it",
 "the motorcyclist's mouth has food in it",
 "the motorcyclist's mouth has popsicle stick in it",
 "the motorcyclist's mouth has cigarette in it"]

In [24]:
labels_per_image = labels[:]

In [25]:
assert len(images) == len(labels_per_image)

In [26]:
BATCH_SIZE = 4

In [27]:
def encode_image_batch(paths):
    imgs = [preprocess(Image.open(p).convert("RGB")) for p in paths]
    imgs = torch.stack(imgs).to(device)
    with torch.no_grad(), torch.autocast(device_type=device if device=="cuda" else "cpu"):
        feats = model.encode_image(imgs)
        feats /= feats.norm(dim=-1, keepdim=True)
    return feats

In [28]:
predictions = {}
all_ranked_labels = []
all_ranked_sims = []
total = len(images)

for start in tqdm(range(0, total, BATCH_SIZE), desc="Processing batches"):
    end = min(start + BATCH_SIZE, total)
    batch_images = images[start:end]
    batch_labels = labels_per_image[start:end]
    batch_image_ids = image_ids[start:end]

    # ---- Encode image features ----
    img_feats = encode_image_batch(batch_images)  # [B, D]
    # print("image encoded", end=',')
    # ---- Encode all text labels (flattened) ----
    text_prompts = []
    img_index_map = []  # to map text → image index

    for i, labels in enumerate(batch_labels):
        for lbl in labels:
            text_prompts.append(lbl)
            img_index_map.append(i)

    text_tokens = tokenizer(text_prompts).to(device)

    with torch.no_grad(), torch.autocast(device_type=device if device=="cuda" else "cpu"):
        text_feats = model.encode_text(text_tokens)
        text_feats /= text_feats.norm(dim=-1, keepdim=True)  # [T, D]
    # print("text encoded")

    # ---- Compute similarities ----
    sims = torch.zeros(len(batch_labels), max(len(lbls) for lbls in batch_labels), device=device)
    for i in range(len(batch_labels)):
        idxs = [j for j, k in enumerate(img_index_map) if k == i]
        sim = img_feats[i].unsqueeze(0) @ text_feats[idxs].T
        sims[i, :len(idxs)] = sim.squeeze(0)

    # ---- Rank labels per image ----
    for i, lbls in enumerate(batch_labels):
        valid_sims = sims[i, :len(lbls)]
        ranked_idx = torch.argsort(valid_sims, descending=True)
        ranked_labels = [lbls[j] for j in ranked_idx.tolist()]
        all_ranked_labels.append(ranked_labels)
        all_ranked_sims.append(valid_sims.tolist())
        predictions[batch_image_ids[i]] = {
            "image_path": batch_images[i],
            "labels": ranked_labels,
            "similarities": valid_sims.tolist(),
            "image_id": batch_image_ids[i]
        }


Processing batches: 100%|██████████| 287/287 [00:59<00:00,  4.80it/s]


In [29]:
len(predictions)

1122

In [30]:
import pickle

output_path = f"/content/drive/MyDrive/DATS 5980/val_predictions_visual_grounding_vitl14.pkl"

with open(output_path, 'wb') as fp:
    pickle.dump(predictions, fp)

print(f"Predictions saved to {output_path}")

Predictions saved to /content/drive/MyDrive/DATS 5980/val_predictions_visual_grounding_vitl14.pkl


In [32]:
for i, (path, ranked) in enumerate(zip(images, all_ranked_labels)):
    print(f"\nImage {i+1} ({path}):")
    for rank, label in enumerate(ranked, 1):
        print(f"  {rank}. {label}")


Image 1 (/content/coco_filtered/val2017/000000461751.jpg):
  1. the motorcyclist's mouth has cigarette in it
  2. the motorcyclist's mouth has toothpick in it
  3. the motorcyclist's mouth has popsicle stick in it
  4. the motorcyclist's mouth has food in it

Image 2 (/content/coco_filtered/val2017/000000377368.jpg):
  1. The number is thirty
  2. The number is one
  3. The number is ten
  4. The number is nine

Image 3 (/content/coco_filtered/val2017/000000563603.jpg):
  1. the pool of water is boiling
  2. the pool of water is dirty
  3. the pool of water is fresh
  4. the pool of water is frozen

Image 4 (/content/coco_filtered/val2017/000000329542.jpg):
  1. top of the cupcakes has butter on it
  2. top of the cupcakes has icing on it
  3. top of the cupcakes has ice cream on it
  4. top of the cupcakes has mayo on it

Image 5 (/content/coco_filtered/val2017/000000182202.jpg):
  1. type of device is a type of mobile phone
  2. type of device is a type of keyboard
  3. type of devi